In [1]:
import re, math
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import nfldb
import sys
import matplotlib as mpl
import seaborn as sns
from __future__ import division
#import psycopg2 as pg
# configure matplotlib plots
%matplotlib inline
mpl.rcParams['font.size'] = 14
mpl.rcParams['figure.figsize'] = [12, 9]#[10, 7.5]
mpl.rcParams['figure.dpi'] = 90
mpl.rcParams['savefig.dpi'] = 100
# configure seaborn plots
sns.set(style="whitegrid", color_codes=True)

yrType = '2016 Regular Season'

In [2]:
db = nfldb.connect()
# get object query nfldb for 2016 season, thru week 9
q = nfldb.Query(db)
q.game(season_year=2016, season_type='Regular')#, week__le=9)
q.sort(('gsis_id', 'asc'))#, ('time', 'asc')])

In [3]:
# functions
def attributeSelect(obj, startExc, endExc='xxx'):
    return [a for a in dir(obj[0]) if not a.startswith(startExc) | a.endswith(endExc)]

def attributeLists(obj, attr, plays=False):
    # list of attributes of object
    #attr = [x for x in dir(obj[0]) if not x.startswith(attrEx)]
    # list of lists of attributes for object
    data = []
    scores = []
    for x in obj:
        data.append([getattr(x, a) for a in attr])
        if plays:
            scores.append(x.score(before=True))
    return data, scores

def icol(df, cols):
    return [i for i, dfCol in enumerate(df.columns) for col in cols if dfCol==col]

def appStrs(cols, string):
    return [c + string for c in cols]

In [4]:
##################################################
##################PLAYS DATAFRAME#################
##################################################
getPlayScore = True

# obtain plays object
playObj = q.as_plays()
# get list of attributes for plays object...
# ...exluding the tuple items below
excludeAttr = ('_', 'kick', 'punt', 'time_', 'xp', 'sql_' , 'defense'
               , 'fill_drives', 'from_id', 'from_row', 'play_p')
attr = attributeSelect(playObj, excludeAttr)
#[a for a in dir(playObj[0]) if not a.startswith(excludeAttr)]
# collect play data into list of lists for each play and its attributes
# ALSO, retrieve score before each play
pLists, score_before_play = attributeLists(playObj, attr, getPlayScore)

# create df with play Lists and with column names from attributes
plays = pd.DataFrame(pLists, columns=attr)
firstRun = True
SHIFTED = False
CUMSUM = False
if getPlayScore:
    # insert score cols
    plays['score'] = score_before_play
    plays[['score_home', 'score_away']] = pd.DataFrame(plays.score.values.tolist())
    plays['score_diff'] = plays.score_home - plays.score_away
    
    #apply(pd.Series)
#playObj = None
#plays.head()

In [5]:
##################################################
#-----------------PLAY EXTRACTION (TARGET)--------------#
##################################################
#pat = ' '.join(map(re.escape, ['pass', 'right']))
playType = ['pass', 'rush']
playDir = ['left', 'up the middle', 'right']
plays['play'] = np.nan
# '[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' accounts for C.Newton & C.Artis-Payne

# exclude plas with illegal forward passes -- noisey play, meant to be pass then run then pass
illPass = plays.description.str.contains('Illegal Forward Pass')
plays = plays.drop(plays[illPass].index)
print('ATTENTION: {0} plays excluded as Illegal Forward Pass'.format(len(illPass[illPass==True])))
playCols = []
for t in playType:
    for d in playDir:
        # new col holder, slash clear col if cell is run more than once
        plays[t+'_'+d[-6:]] = 0
        # regular expression to search for pass
        # --FLAWED AT .*?-- reExp = r'[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' + re.escape(t) + '\s.*?\s' + re.escape(d[-6:])
        
        # looong regEx to search for  pass
        # reExp = r'[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' + re.escape(t) + '\s[a-z]\s' + re.escape(d[-6:]) +#eg C.Newton pass short right
        # '|' '[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' + re.escape(t) +  '\s[a-z][a-z]\s' + re.escape(d[-6:]) +#eg C.Newton pass incomlete short right
        # '|' '[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' + re.escape(t) +  '\s' + re.escape(d[-6:]) #eg C.Newton pass right
        reExp = r'[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' + re.escape(t) + '\s[a-z]+\s' + re.escape(d[-6:]) + '|' '[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' + re.escape(t) +  '\s[a-z]+\s[a-z]+\s' + re.escape(d[-6:]) + '|' '[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' + re.escape(t) +  '\s' + re.escape(d[-6:]) 
    
        # regular expression to search for rush
        # to account for the fact that 'rush' is only sometimes included in description...
        # ...other times just a space e.g., K.Moreno right tackle for 8 yards
        if t=='rush':
            # if labeling QB scrambles as run insert: 
            # + '|' + [A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' + '\s'.join(re.escape('scramble'), re.escape(d))
            reExp = r'[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s' + re.escape(t) + '\s' + '|'.join([re.escape(d), '[A-Z]\.[A-Z][a-z]+\-?[a-z]+\s']) + re.escape(d)
        
        mask = plays.description.str.contains(reExp, regex=True, flags=re.I)
        # single col of play categories
        plays.loc[mask, 'play'] = t + '_' + d[-6:]
        # dummy col each target
        plays.loc[mask, t+'_'+d[-6:]] = 1
        playCols.append(t+'_'+d[-6:])
        print('{0} {1} count: {2}'.format(t, d[-6:], plays[t+'_'+d[-6:]].sum()))
plays.play = plays.play.astype('category')
plays.groupby('play').size()

ATTENTION: 7 plays excluded as Illegal Forward Pass
pass left count: 7008
pass middle count: 4494
pass right count: 7771
rush left count: 4515
rush middle count: 3546
rush right count: 4435


play
pass_left      7008
pass_middle    4494
pass_right     7771
rush_left      4515
rush_middle    3546
rush_right     4435
dtype: int64

In [6]:
##################################################
##################GAMES DATAFRAME#################
##################################################
# obtain games object
gameObj = q.as_games()

# get list of attributes for games object...
# ...exluding the tuple items below
excludeAttr = ('_', 'from_', 'time_', 'sql_' , 'away_turnovers', 'home_turnovers'
               , 'winner', 'score_', 'plays', 'play_p', 'players', 'away_s', 'home_s', 'drives', 'loser'
              , 'gamekey')
attr = attributeSelect(gameObj, excludeAttr, 'score')

# construct list of lists for each game and attribute
# from game object attributes
gamesList, no = attributeLists(gameObj, attr)

# games dataframe 
games = pd.DataFrame(gamesList, columns=attr)
print(games.head())
games.columns

  away_team day_of_week finished     gsis_id home_team is_playing season_type  \
0       CAR    Thursday     True  2016090800       DEN      False     Regular   
1        TB      Sunday     True  2016091100       ATL      False     Regular   
2       BUF      Sunday     True  2016091101       BAL      False     Regular   
3       CHI      Sunday     True  2016091102       HOU      False     Regular   
4        GB      Sunday     True  2016091103       JAC      False     Regular   

   season_year                 start_time  week  
0         2016  2016-09-08 20:30:00-04:00     1  
1         2016  2016-09-11 13:00:00-04:00     1  
2         2016  2016-09-11 13:00:00-04:00     1  
3         2016  2016-09-11 13:00:00-04:00     1  
4         2016  2016-09-11 13:00:00-04:00     1  


Index([u'away_team', u'day_of_week', u'finished', u'gsis_id', u'home_team',
       u'is_playing', u'season_type', u'season_year', u'start_time', u'week'],
      dtype='object')

In [7]:
##################################################
##################DRIVES DATAFRAME#################
##################################################
# obtain drives object
driveObj = q.as_drives()

# get list of attributes for drives object...
# ...exluding the tuple items below
excludeAttr = ('_', 'fill', 'score', 'from_', 'time_', 'sql_' , 'end_time', 'play_p', 'plays', 'pos_team', 'penalty_yards')
attr = attributeSelect(driveObj, excludeAttr)
print(attr)
# construct list of lists for each game and attribute
# from game object attributes
dList, no = attributeLists(driveObj, attr)

# games dataframe 
drives = pd.DataFrame(dList, columns=attr)
print(drives.head())
drives.columns

['drive_id', 'end_field', 'first_downs', 'game', 'gsis_id', 'play_count', 'pos_time', 'result', 'start_field', 'start_time', 'yards_gained']
   drive_id end_field  first_downs  \
0        21    OWN 40            0   
1        20    OPP 32            3   
2        19    OWN 18            0   
3        18    OPP 18            3   
4        17     OPP 1            3   

                                                game     gsis_id  play_count  \
0  Regular 2016 week 1 on 09/08 at 08:30PM, CAR (...  2016090800           2   
1  Regular 2016 week 1 on 09/08 at 08:30PM, CAR (...  2016090800          18   
2  Regular 2016 week 1 on 09/08 at 08:30PM, CAR (...  2016090800           6   
3  Regular 2016 week 1 on 09/08 at 08:30PM, CAR (...  2016090800          11   
4  Regular 2016 week 1 on 09/08 at 08:30PM, CAR (...  2016090800          13   

  pos_time       result start_field start_time  yards_gained  
0    00:04  End of Game      OWN 40   Q4 00:04            -1  
1    03:02    Missed FG

Index([u'drive_id', u'end_field', u'first_downs', u'game', u'gsis_id',
       u'play_count', u'pos_time', u'result', u'start_field', u'start_time',
       u'yards_gained'],
      dtype='object')

In [8]:
# check for duplicate columns
#print(plays.columns[plays.columns.duplicated()])
plays.columns.duplicated().max()

False

In [9]:
## -------MERGE PLAYS WITH DRIVES and GAMES DF-----------
plays = plays.merge(drives, how='left')
plays = plays.merge(games, how='left', on='gsis_id')
# flip score diff for away teams, to make accurate
plays.loc[plays.pos_team == plays.away_team, 'score_diff'] = plays.loc[plays.pos_team == plays.away_team, 'score_diff'] * -1
# plays[['score', 'score_diff', 'pos_team', 'home_team', 'play']].tail(50)
# free memory space
drives = None
games = None
driveObj = None
gameObj = None
#playObj = None

In [27]:
plays.iloc[513].description

'(12:48) (Shotgun) B.Osweiler scrambles up the middle to CHI 36 for 15 yards (J.Glenn).'

In [10]:
##################################################
#-----------------INTEGRITY CHECK--------------#
##################################################
print('\n{0} - Estimated total number of usuable plays in nfldb, with current selection setup\n'.format(plays[plays.play.notnull()].shape[0]*10))
#------------CHECK OF EXCLUDED PLAYS-----------#
exPlays = plays.loc[plays.play.isnull(), ['description', 'gsis_id', 'drive_id']]
exPlays.head(5)
randIdx = np.random.randint(0, exPlays.shape[0], 10)
print('\nExcluded Plays (randomly selected from set)\n')
print(exPlays.iloc[randIdx].description)

#######-------CHECK FOR OVERLAPING PLAY LABELS-----###########
all_overlap = plays[(plays.rush_left+plays.rush_middle+plays.rush_right+plays.pass_left+plays.pass_middle+plays.pass_right)>1].description
pass_overlap = plays[(plays.pass_left+plays.pass_middle+plays.pass_right)>1].description
pass_exclusive = plays[(plays.pass_left+plays.pass_middle+plays.pass_right)==1].description
#print(all_overlap.shape[0]-pass_overlap.shape[0])
#print(plays[(plays.rush_right+plays.pass_right)>1].description)

if all_overlap.shape[0]==0:
    print('\nNo Overalapping Play Labels Present!!\n')
else:
    print('\nATTENTION: Overlapping Play lables exist, examples:\n')
    print(all_overlap[:5])

#######-------Number of QB scrambles runs-----###########
scramble_n = plays[plays.description.str.contains('scrambles')].shape[0]
print('\n{0} QB scrambles\n'.format(scramble_n))
#plays[plays.description.str.contains('scrambles')].description


#######-------PASSES with NO Direction in description-----###########
noDirPass = exPlays[exPlays.description.str.contains('pass')]
print('\n{0} Passes with No Direction description\n'.format(noDirPass.shape[0]))
print('\nExample of Pass with no Direction description:')
noDirPass.iloc[np.random.randint(0, noDirPass.shape[0], 3)].description #idx: 21266, 4525


317690 - Estimated total number of usuable plays in nfldb, with current selection setup


Excluded Plays (randomly selected from set)

10888                          Timeout #1 by DAL at 00:44.
2000     R.Bullock kicks 65 yards from NYG 35 to end zo...
22617    (2:18) (Punt formation) M.Darr punts 34 yards ...
22368       (:55) M.Sanchez kneels to CLE 17 for -1 yards.
14125    D.Carpenter extra point is GOOD, Center-G.Sanb...
13727    (7:01) A.Lee punts 39 yards to TB 21, Center-J...
7213     (1:55) D.Hopkins 37 yard field goal is GOOD, C...
13250    G.Zuerlein kicks 65 yards from LA 35 to end zo...
11408    (6:34) (No Huddle, Shotgun) B.Gabbert scramble...
34215    (9:36) M.Palardy punts 46 yards to end zone, C...
Name: description, dtype: object

No Overalapping Play Labels Present!!


692 QB scrambles


121 Passes with No Direction description


Example of Pass with no Direction description:


31215    TWO-POINT CONVERSION ATTEMPT. A.Smith pass to ...
25963    (Pass formation) TWO-POINT CONVERSION ATTEMPT....
14853    END GAME NE 12-Brady 47th career game with 3+ ...
Name: description, dtype: object

In [11]:
# create seperate play type and play direction labels
plays[['play_type', 'play_dir']] = plays.play.str.split('_').apply(pd.Series)
# create dummy vectors for cummulated proportions
plays = pd.concat([plays,
                   pd.get_dummies(plays['play_type'], prefix='type'),
                   pd.get_dummies(plays['play_dir'], prefix='dir')], axis=1)

In [12]:
plays.head()

,description,down,drive,drive_id,first_down,fourth_down_att,fourth_down_conv,fourth_down_failed,fumbles_forced,fumbles_lost,...,season_year,start_time_y,week,play_type,play_dir,type_pass,type_rush,dir_left,dir_middle,dir_right
0,"G.Gano kicks 65 yards from CAR 35 to end zone,...",NaN,[Fumble ] DEN from OWN 25 to OPP 29 (last...,1,0,0,0,0,0,0,...,2016,2016-09-08 20:30:00-04:00,1,NaN,NaN,0,0,0,0,0
1,(15:00) T.Siemian pass short left to D.Thomas ...,1.0,[Fumble ] DEN from OWN 25 to OPP 29 (last...,1,1,0,0,0,0,0,...,2016,2016-09-08 20:30:00-04:00,1,pass,left,1,0,1,0,0
2,(14:17) T.Siemian pass incomplete short right ...,1.0,[Fumble ] DEN from OWN 25 to OPP 29 (last...,1,0,0,0,0,0,0,...,2016,2016-09-08 20:30:00-04:00,1,pass,right,1,0,0,0,1
3,(14:13) T.Siemian pass incomplete short right ...,2.0,[Fumble ] DEN from OWN 25 to OPP 29 (last...,1,0,0,0,0,0,0,...,2016,2016-09-08 20:30:00-04:00,1,pass,right,1,0,0,0,1
4,(14:08) (Shotgun) T.Siemian pass short left to...,3.0,[Fumble ] DEN from OWN 25 to OPP 29 (last...,1,1,0,0,0,0,0,...,2016,2016-09-08 20:30:00-04:00,1,pass,left,1,0,1,0,0


In [13]:
# exclude rows for non-selected plays (null plays)
plays = plays[plays.play.notnull()]

# cum sum stats columns
col2cum = [c for c in plays.columns for x in ['type', 'dir', 'first', 'third', 'fourth', 'penalty', 'offense', 'passing', 'receiving', 'rushing'] if c.startswith(x) and not c.endswith('downs')]
gcumCols = appStrs(col2cum, '_cum_gm')
dcumCols = appStrs(col2cum, '_cum_dr')
# multilevel index and sort the index levels for cumsum and shift below
plays = plays.set_index(['gsis_id', 'pos_team', 'drive_id']).sortlevel()
# plays.iloc[34:74, icol(plays, dcumCols)]
# icol(plays, dcumCols)

In [14]:
col2cum

['first_down',
 'fourth_down_att',
 'fourth_down_conv',
 'fourth_down_failed',
 'offense_tds',
 'offense_yds',
 'passing_att',
 'passing_cmp',
 'passing_cmp_air_yds',
 'passing_first_down',
 'passing_incmp',
 'passing_incmp_air_yds',
 'passing_int',
 'passing_sk',
 'passing_sk_yds',
 'passing_tds',
 'passing_twopta',
 'passing_twoptm',
 'passing_twoptmissed',
 'passing_yds',
 'penalty',
 'penalty_first_down',
 'penalty_yds',
 'receiving_rec',
 'receiving_tar',
 'receiving_tds',
 'receiving_twopta',
 'receiving_twoptm',
 'receiving_twoptmissed',
 'receiving_yac_yds',
 'receiving_yds',
 'rushing_att',
 'rushing_first_down',
 'rushing_loss',
 'rushing_loss_yds',
 'rushing_tds',
 'rushing_twopta',
 'rushing_twoptm',
 'rushing_twoptmissed',
 'rushing_yds',
 'third_down_att',
 'third_down_conv',
 'third_down_failed',
 'type_pass',
 'type_rush',
 'dir_left',
 'dir_middle',
 'dir_right']

In [15]:
# checker to avoid Accumulating twice
if not CUMSUM:
    print('Accumulating Stats...')
    dfc = None
    dfc = plays[col2cum]
    # duplicate cols to cum
    dfc[dcumCols] = dfc[col2cum]
    dfc[gcumCols] = dfc[col2cum]
    cumCols = dcumCols + gcumCols

    ##------- Cumulate Stats across Plays for each Team and separately for each Drive within a Game -----##
    dc = dfc.groupby(level=[0, 1, 2])[dcumCols].cumsum()
    ###------- Cumulate Stats across Plays for each Team over drives within a Game -----###
    gc = dfc.groupby(level=[0, 1])[gcumCols].cumsum()
    #update checker
    CUMSUM = True
    #plays = plays.drop(cumCols, axis=1)
    dfc = None
    #print(dc.iloc[35:75, :])

Accumulating Stats...


/home/jsync/anaconda3/envs/py2/lib/python2.7/site-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [16]:
# checker to avoid shifting and NaNing twice
if not SHIFTED:
    print('Shifting Stats...')
    # concatenate rows of plays and cumsum dfs
    df = None
    print(plays.shape, dc.shape, gc.shape)
    df = pd.concat([plays, dc, gc], axis=1)
    print(icol(plays, dcumCols))
    print(df.shape)
    col2shift = col2cum + dcumCols + gcumCols
    # shift selected rows down 1, so that current play stats aren't leaked into the model
    dfs = df[col2shift].shift(1)
    
    # NaN For drive cum cols: Set 1st play of each teams drives to prevent accumulating stats across teams AND drives
    dfs.iloc[plays.groupby(level=[0, 1, 2]).size().cumsum()[:-1], icol(dfs, dcumCols)] = np.nan
    # NaN For game cum cols: Set 1st play of each team to prevent accumlating stats across teams
    dfs.iloc[plays.groupby(level=[0, 1]).size().cumsum()[:-1], icol(dfs, gcumCols)] = np.nan
    dfs.iloc[plays.groupby(level=[0, 1]).size().cumsum()[:-1], icol(dfs, col2cum)] = np.nan
   
    #update checker
    SHIFTED = True
    df = None
    #print(dfs.iloc[35:75, icol(dfd, dcumCols)])

Shifting Stats...
((31769, 99), (31769, 48), (31769, 48))
[]
(31769, 195)


In [17]:
# COMBINE shifted columns into main df - plays
# prevent column name overlap when concatenating shifted dfs with plays
# rename individual stats columns with appended '_sh1' - shifted 1 play forward
dfs.rename(index=str, columns=dict(zip(col2cum, appStrs(col2cum, '_sh1'))), inplace=True)
df = pd.concat([plays.reset_index(), dfs.reset_index()], axis=1)
plays, dfs = None, None
plays = df
df = None
plays.shape

(31769, 249)

In [18]:
# Remove duplicate columns
print('Duplicate column names:\n')
# duplicate columns boolean
dupCol = plays.columns.duplicated()
plays = plays.iloc[:, [i for i, c in enumerate(plays.columns) if not dupCol[i]]]
duplicateCols = plays.columns[dupCol]
print(duplicateCols)

Duplicate column names:

Index([u'first_down_sh1', u'fourth_down_att_sh1', u'fourth_down_conv_sh1'], dtype='object')


/home/jsync/anaconda3/envs/py2/lib/python2.7/site-packages/pandas/indexes/base.py:1434: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 246 but corresponding boolean dimension is 249
  result = getitem(key)


In [30]:
# extract quarter # and game clock time and produce their own columns
plays['time'] = plays.time.astype('str')
plays[['quarter', 'qtr_time']] = plays['time'].apply(lambda x: x.split(' ')).apply(pd.Series)

In [36]:
# adjust quarter and game clock data types, ordinal and time respectively
quarter_mapping = {'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4, 'OT': 5}

# numeric quarter codes
plays['quarter_code'] = plays.quarter.map(quarter_mapping)
plays.quarter_code.unique()

array([1, 2, 3, 4, 5])

In [37]:
## make time object
plays.qrt_time = pd.to_datetime(plays.qtr_time, format='%M:%S').dt.time
plays[['quarter', 'qtr_time']].dtypes

quarter     category
qtr_time      object
dtype: object

In [39]:
# convert quarter time to min and sec floats for machine learning
plays[['qtr_min', 'qtr_sec']] = plays.qtr_time.apply(lambda x: x.split(':')).apply(pd.Series).astype('float')
plays.qtr_sec /= 60

In [40]:
# create full quarter time
plays['qtr_timef'] = plays.qtr_min + plays.qtr_sec
plays['qtr_timef']

0        11.616667
1        11.016667
2        10.350000
3         9.650000
4         9.583333
5         8.916667
6         8.116667
7         7.333333
8         1.050000
9         0.916667
10        0.200000
11       12.150000
12       11.466667
13       11.333333
14       10.700000
15       10.100000
16        9.416667
17        8.800000
18        8.200000
19        7.566667
20        6.883333
21        6.833333
22        6.616667
23        5.983333
24        5.916667
25        5.283333
26        4.683333
27        4.000000
28        3.116667
29        1.750000
           ...    
31739     8.550000
31740     7.916667
31741     6.750000
31742     6.250000
31743     5.666667
31744     5.583333
31745     5.033333
31746     2.866667
31747     2.216667
31748     0.466667
31749     0.133333
31750     0.066667
31751    11.633333
31752    10.933333
31753     8.083333
31754     7.466667
31755     6.866667
31756     4.983333
31757     4.550000
31758     3.983333
31759    14.133333
31760    13.

In [ ]:
# remove colon from timezone
plays.start_time_y = plays.start_time_y.apply(lambda x: x[:-3] + '00')

In [ ]:
# get game start time, west coast teams struggle with 1 pm EST kickoff
plays['start_time_gm'] = pd.to_datetime(plays.start_time_y, format='%Y-%m-%d %H:%M:%S').dt.time.astype('str')
plays[['st_hr', 'st_min', 'st_sec']] = plays.start_time_gm.str.split(':').apply(pd.Series)#.astype('float')
plays['st_tz'] = plays.start_time_y.apply(lambda x: x[-3:-2])

In [ ]:
plays.start_time_gm.unique()

In [ ]:
plays['start_timef_gm'] = plays.st_hr.astype(int) + (plays.st_min.astype(int) / 60)
plays.start_timef_gm[:5]

In [ ]:
# home field advantage feature; 1 = home team
plays.loc[plays.home_team==plays.pos_team, 'home'] = 1
plays.home = plays.home.fillna(0).astype('int')

In [ ]:
# EXTRACT side of FIELD and YARD number from YARDLINE
plays[['field', 'yard']] = plays.yardline.str.split(' ').apply(pd.Series)

In [ ]:
# convert yard values for rescaling below
plays.loc[plays.field=='MIDFIELD', 'yard'] = 50
plays.yard = plays.yard.astype('int')
plays['yardfield'] = plays.yard

# scale yardline from 1-99.
# yardline 49-1 in your on own side of the field are 51-99.
# i.e., you're farther away from your scoring endzone (the opponent's endzone)
plays.loc[plays.field=='OWN', 'yardfield'] = plays.loc[plays.field=='OWN', 'yardfield'].apply(lambda x: 50 + (50 - x))

In [ ]:
# assign feature for team on defense
plays.loc[plays.pos_team==plays.home_team, 'def_team'] = plays.loc[plays.pos_team==plays.home_team, 'away_team']
plays.loc[plays.pos_team==plays.away_team, 'def_team'] = plays.loc[plays.pos_team==plays.away_team, 'home_team']

In [ ]:
# create dummy vectors for categorical dummy features
plays = pd.concat([plays, pd.get_dummies(plays['pos_team'], prefix='off')], axis=1)
plays = pd.concat([plays, pd.get_dummies(plays['def_team'], prefix='def')], axis=1)
plays = pd.concat([plays, pd.get_dummies(plays['day_of_week'], prefix='day')], axis=1)
plays.head()

In [41]:
# SAVE to csv
plays.to_csv('data/2016_reg_plays_gd.csv', index=False)

In [ ]:
# incoporate new features and dummy cols in features array
adfeat = ['quarter_code', 'qtr_timef', 'down', 'yards_to_go', 'score_diff', 'week', 'home', 'yardfield', 'start_timef_gm']
dumPrefix = ['off_', 'def_', 'day_']
catExc = ['pos_team', 'def_team', 'day_of_week']
dumCols = [c for c in plays.columns for d in dumPrefix if c.startswith(d) and c not in catExc]
features = cumCols + prevPlay + adfeat + dumCols
features

In [ ]:
excFeatures = ['receiving_tar_sh1',
               'receiving_tar_cum_gm',
               'receiving_tar_cum_dr',
               'receiving_yds_sh1',
               'receiving_yds_cum_gm',
               'receiving_yds_cum_dr',
               'rushing_first_down_cum_dr', 
               'offense_tds_sh1', 
               'third_down_att_sh1', 
               'fourth_down_conv_cum_dr', 
               'fourth_down_conv_cum_gm', 
               'third_down_failed_cum_dr', 
               'passing_cmp_air_yds_cum_gm', 
               'third_down_failed_sh1', 
               'third_down_failed_cum_dr']
features = [feat for feat in features if feat not in excFeatures]
features = [f for f in features if 'twopt' not in f]
print(len(features))
features

In [ ]:
np.save('data/features_dum', features)